Task1_0603. train.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

1. 레이블인코딩탐색 : Accuracy: 0.8156424581005587
- Age 결측치 : Sex 별 평균으로 처리
- FamilySize = df.SibSp + df.Parch + 1
- Embarked & Sex 레이블 인코딩
  - embarked 레이블 인코딩시 nan값 자동처리됨.

2. 원핫 인코딩 탐색 : Accuracy: 0.8268156424581006
- FamilySize = df.SibSp + df.Parch + 1
- Embarked & Sex 원핫 인코딩
  - Embarked 결측치 S로 수정시 0.8268156424581006
  - Embarked 결측치 Q로 수정시 0.8268156424581006
  - Embarked 결측치 C로 수정시 0.8268156424581006
    - Embarked 결측치 2개는 의미없는걸로

3. FamilySize 대신 HasFamily 설정시 : Accuracy: 0.8156424581005587
4. FamilySize & HasFamily : Accuracy: 0.8324022346368715
  - 가족수 & 가족의 존재여부가 생존률에 영향을 미치는가

5. Age 컬럼을 AgeGroup으로 범주화 : Accuracy: 0.8212290502793296
6. Age & AgeGroup : Accuracy: 0.8268156424581006

7. 남/여 별 나이 평균 : Accuracy: 0.8156424581005587
8. 가족 수 별 성별의 상관계수 : 0.8156424581005587
9. 가족 별 남성 비율 : Accuracy: 0.8379888268156425

10. Fare 10개 그룹으로 범주화 : Accuracy: 0.7821229050279329
11. 정박지 별 객실 요금 평균 : Accuracy: 0.8156424581005587
12. Fare를 10개 범주로 확대하여 구간별 남녀성비 : Accuracy: 0.8212290502793296

In [42]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
# 파일불러오기
file_path = 'drive/MyDrive/KDT_2404/m3_분석 라이브러리/dataset/train.csv'
df = pd.read_csv(file_path)

# 결측값이 많은 컬럼, 필요없는 컬럼 삭제
df.drop(columns=['PassengerId', 'Pclass', 'Ticket', 'Cabin', 'Name'], inplace=True)

# Embarked 결측값 처리
df['Embarked'].fillna('C', inplace=True)

# Age 결측치 : Sex 별 평균으로 결측치 처리
age_mean_by_sex = round(df.groupby('Sex').Age.transform('mean'))
df['Age'].fillna(age_mean_by_sex, inplace=True)

# 가족 수
df['FamilySize'] = df.SibSp + df.Parch + 1

# 가족의 존재 여부
df['HasFamily'] = df.SibSp + df.Parch > 0
df.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# Embarked & Sex ONEHOT 인코딩
onehot_encoder = OneHotEncoder(sparse=False)
columns_to_encode = df[['Embarked', 'Sex']]
encoded_data = onehot_encoder.fit_transform(columns_to_encode)
encoded_df = pd.DataFrame(encoded_data, columns=onehot_encoder.get_feature_names_out(['Embarked', 'Sex']))
df = pd.concat([df, encoded_df], axis=1)
df.drop(['Embarked', 'Sex'], axis=1, inplace=True)

# 가족 별 남성 비율
df['MalebyFamily'] = df['Sex_male'] / df['FamilySize']

In [45]:
# 타겟 변수와 피처 변수 설정
X = df.drop('Survived', axis=1)
y = df['Survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred)) # 예측 정확도
#print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))


(712, 10) (179, 10) (712,) (179,)
Accuracy: 0.8379888268156425
